# III. Arrival Process Simulation

In [6]:
import numpy as np
import clearing_price
from matplotlib import pyplot as plt
import scipy as sp
from agent_manager import AgentManager
import pandas as pd

# import warnings
# from pandas.errors import SettingWithCopyWarning
# warnings.simplefilter(action='ignore', category=[SettingWithCopyWarning, FutureWarning])

In [7]:
am = AgentManager()

TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union

In [ ]:
# Parameters
time_horizon = 1 # years
time_step = 1/12 # month

expected_buyer_arrivals = 50000/1000 # 1000s of people / year
expected_seller_arrivals = 6000/1000 # 1000s of people / year

time_discounting = lambda future_time : 1/(1 + np.log(1/0.98) * future_time) # hyperbolic discounting
max_discounted_time = 100

buyer_fee = lambda : np.random.normal(0,0.1) # dollars of fees and costs buyer must pay to transplant
seller_fee = lambda : np.random.normal(0,0.1) # dollars of fees and costs seller must pay to transplant
seller_transplant_mortality_fxn = lambda mort : mort * 1.1
seller_transplant_income_fxn  = lambda inc : inc * 0.9

# A dynamically growing array that tracks the buyers entering the market
buyers = pd.DataFrame(columns=["AGE", "IS_MALE", "IS_WHITE", "IS_BLACK", "IS_HISPANIC", "GFR", "HAS_DIABETES", "PRIOR_TRANSPLANT", "WORK_INCOME_TCR", "ATTENDED_COLLEGE", "ENTRANCE_TIME", "TRANSPLANT_TIME", "DEATH_TIME", "VALUATION" , "PRICE"]).astype(float)

# A dynamically growing array that tracks the sellers entering the market
sellers = pd.DataFrame(columns=["AGE", "IS_MALE", "IS_WHITE", "IS_BLACK", "IS_HISPANIC", "EDUC", "HAS_DIABETES", "BMI", "HEIGHT", "ENTRANCE_TIME", "TRANSPLANT_TIME", "DEATH_TIME", "VALUATION", "PRICE"]).astype(float)

# The main discrete-time simulation loop
total_steps = int(time_horizon/time_step)
for step in range(total_steps):
    curr_time = step*time_step
    print(f"Analyzing time {curr_time}, step {step} of {total_steps}...")

    # update ages of all buyers and sellers
    buyers["AGE"] += time_step
    sellers["AGE"] += time_step

    # a poisson distribution number of buyers and sellers arrive
    for i in range(np.random.poisson(lam=expected_buyer_arrivals*time_step)):
        new_buyer = am.generate_recipient()
        new_buyer["ENTRANCE_TIME"] = step*time_step
        buyers = buyers.append(new_buyer)
    for i in range(np.random.poisson(lam=expected_seller_arrivals*time_step)):
        new_seller = am.generate_recipient()
        new_seller["ENTRANCE_TIME"] = step*time_step
        sellers = sellers.append(new_seller)
    
    # calculate valuation for each buyer and seller that hasn't yet transplanted or died
    # integrate income * mortality * time_discounting
    looking_buyers_indices = (np.isnan(buyers["TRANSPLANT_TIME"]) & np.isnan(buyers["DEATH_TIME"]))
    for agent_index in np.where(looking_buyers_indices)[0]:
        agent = buyers.iloc[agent_index]
        buyers.iloc[agent_index]["VALUATION"] = sp.integrate.quad( # discounted future income conditional on transplant
            lambda future_time : am.recipient_income_transplant(agent, future_time, agent["AGE"]+future_time) * am.recipient_mortality_transplant(agent, curr_time-agent["ENTRANCE_TIME"], agent["AGE"]+future_time) * time_discounting(future_time),
            0, max_discounted_time
        ) - sp.integrate.quad( # discounted future income conditional on dialysis
            lambda future_time : am.recipient_income_dialysis(agent, agent["AGE"]+future_time) * am.receipient_mortality_dialysis(agent, agent["AGE"]+future_time) * time_discounting(future_time),
            0, max_discounted_time
        ) - buyer_fee
    looking_seller_indices = (np.isnan(sellers["TRANSPLANT_TIME"]) & np.isnan(sellers["DEATH_TIME"]))
    for agent_index in np.where(looking_seller_indices)[0]:
        agent = sellers.iloc[agent_index]
        sellers.iloc[agent_index]["VALUATION"] = sp.integrate.quad( # discounted future income conditional on transplant
            lambda future_time : seller_transplant_income_fxn(am.giver_income(agent, agent["AGE"]+future_time)) * seller_transplant_mortality_fxn(am.giver_mortality(agent, agent["AGE"]+future_time)) * time_discounting(future_time),
            0, max_discounted_time
        ) - sp.integrate.quad( # discounted future income conditional on dialysis
            lambda future_time : am.giver_income(agent, agent["AGE"]+future_time) * am.giver_mortality(agent, agent["AGE"]+future_time) * time_discounting(future_time),
            0, max_discounted_time
        )
    
    # clear the market
    if looking_buyers_indices.shape[0] > 0 and looking_seller_indices.shape[0] > 0:
        price = clearing_price.find_median_clearing_price(buyers[looking_buyers_indices]["VALUATION"], sellers[looking_seller_indices]["VALUATION"])
        if price:
            successful_buyer_indices = looking_buyers_indices & (buyers["VALUATION"] > price)
            buyers[successful_buyer_indices]["TRANSPLANT_TIME"] = curr_time
            buyers[successful_buyer_indices]["PRICE"] = price
            successful_seller_indices = looking_seller_indices & (sellers["VALUATION"] < price)
            sellers[successful_seller_indices]["TRANSPLANT_TIME"] = curr_time
            sellers[successful_seller_indices]["PRICE"] = price
        
        # kill off buyers and sellers according to mortality rate
        for living_buyer_index in np.where(np.isnan(buyers["DEATH_TIME"]))[0]:
            agent = buyers.iloc[living_buyer_index]
            if not np.isnan(agent["TRANSPLANT_TIME"]): mort = am.recipient_mortality_transplant(agent, agent["TRANSPLANT_TIME"]-agent["ENTRANCE_TIME"], agent["AGE"])
            else: mort = am.recipient_income_dialysis(agent, agent["AGE"])
            if np.random.random() < mort: buyers[living_buyer_index]["DEATH_TIME"] = curr_time
        for living_seller_index in np.where(np.isnan(sellers["DEATH_TIME"]))[0]:
            agent = sellers.iloc[living_seller_index]
            if not np.isnan(agent["TRANSPLANT_TIME"]): mort = seller_transplant_mortality_fxn(am.giver_mortality(agent, agent["TRANSPLANT_TIME"]-agent["ENTRANCE_TIME"], agent["AGE"]))
            else: mort = am.giver_mortality(agent, agent["TRANSPLANT_TIME"]-agent["ENTRANCE_TIME"], agent["AGE"])

Analyzing time 0.0, step 0 of 12...


TypeError: issubclass() arg 2 must be a class, a tuple of classes, or a union